In [ ]:
print ("بسم الله والحمدلله والصلاة والسلام على رسول الله صلى الله عليه وسلم")

بسم الله والحمدلله والصلاة والسلام على رسول الله صلى الله عليه وسلم


In [ ]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar
from tabulate import tabulate

#McNemar Test For Majority Voting

In [ ]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar
from tabulate import tabulate

def load_predictions(file_path):
    """
    Reads a .txt file with fixed headers:
    Token, Gold Tag, Predicted Tag
    Handles malformed lines safely.
    """
    print(f"\n📂 Loading: {file_path}")
    try:
        df = pd.read_csv(
            file_path,
            sep="\t",
            comment="#",
            skip_blank_lines=True,
            engine="python",
            on_bad_lines="warn"
        )
    except Exception as e:
        raise RuntimeError(f"❌ Error reading {file_path}: {e}")

    expected_cols = ["Token", "Gold Tag", "Predicted Tag"]
    if len(df.columns) > 3:
        print(f"⚠️ {file_path}: found {len(df.columns)} columns, trimming to 3.")
        df = df.iloc[:, :3]
        df.columns = expected_cols
    elif len(df.columns) == 3:
        df.columns = expected_cols
    else:
        raise ValueError(f"❌ {file_path} has invalid column structure ({len(df.columns)} columns).")

    before = len(df)
    df = df.dropna()
    after = len(df)
    if before != after:
        print(f"⚠️ Dropped {before - after} rows with missing values in {file_path}.")
    print(f"✅ Loaded {len(df)} valid rows from {file_path}")
    return df


# ==== الملفات ====
baseline_files = [
    "/content/predictions_ANERtestC10_Camel-ca-ner.txt",
    "/content/predictions_ANERtestC10_Camel-msa-ner.txt",
    "/content/predictions_ANERtestC10_Camel-mix-ner.txt",
    "/content/predictions_ANERtestC10_Multilingual2.txt"
]
ensemble_file = "/content/Compined_Predictions_Majority_ANER.txt"


# ==== تحميل البيانات ====
ensemble = load_predictions(ensemble_file)

results_summary = []
tables = {}

# ==== اختبار McNemar ====
for file in baseline_files:
    baseline = load_predictions(file)

    # توحيد البيانات حسب Token لضمان التطابق
    merged = pd.merge(
        baseline,
        ensemble,
        on="Token",
        suffixes=("_base", "_ens"),
        how="inner"
    )

    print(f"🔄 Merged {len(merged)} rows between {file.split('/')[-1]} and ensemble")

    baseline_correct = merged["Gold Tag_base"] == merged["Predicted Tag_base"]
    ensemble_correct = merged["Gold Tag_ens"] == merged["Predicted Tag_ens"]

    a = sum(baseline_correct & ensemble_correct)
    b = sum(baseline_correct & ~ensemble_correct)
    c = sum(~baseline_correct & ensemble_correct)
    d = sum(~baseline_correct & ~ensemble_correct)

    table = [[a, b], [c, d]]
    tables[file] = table

    result = mcnemar(table, exact=False, correction=True)
    significance = "Significant (p < 0.05)" if result.pvalue < 0.05 else "Not significant"

    results_summary.append([
        file.replace(".txt", "").split("/")[-1],
        result.statistic,
        result.pvalue,
        significance
    ])

# ==== طباعة النتائج ====
print("\n=== Statistical Significance Results (McNemar Test) ===\n")
headers = ["Baseline Model", "Chi² Statistic", "p-value", "Significance"]
print(tabulate(results_summary, headers=headers, tablefmt="grid", floatfmt=".5f"))

print("\n=== Detailed 2x2 Comparison Tables ===")
for file, table in tables.items():
    print(f"\n{file.replace('.txt', '').split('/')[-1]}:")
    print(tabulate(
        table,
        headers=["Ensemble Correct", "Ensemble Wrong"],
        showindex=["Baseline Correct", "Baseline Wrong"],
        tablefmt="fancy_grid"
    ))



📂 Loading: /content/Compined_Predictions_Majority_ANER.txt
⚠️ Dropped 2 rows with missing values in /content/Compined_Predictions_Majority_ANER.txt.
✅ Loaded 14235 valid rows from /content/Compined_Predictions_Majority_ANER.txt

📂 Loading: /content/predictions_ANERtestC10_Camel-ca-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-ca-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-ca-ner.txt
🔄 Merged 685599 rows between predictions_ANERtestC10_Camel-ca-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-msa-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-msa-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-msa-ner.txt
🔄 Merged 685608 rows between predictions_ANERtestC10_Camel-msa-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-mix-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERte

/tmp/ipython-input-4068105385.py:13: ParserWarning: Skipping line 4746: Expected 3 fields in line 4746, saw 4

  df = pd.read_csv(
/tmp/ipython-input-4068105385.py:13: ParserWarning: Skipping line 13249: Expected 3 fields in line 13249, saw 5

  df = pd.read_csv(



=== Statistical Significance Results (McNemar Test) ===

+---------------------------------------+------------------+-----------+------------------------+
| Baseline Model                        |   Chi² Statistic |   p-value | Significance           |
+=======================================+==================+===========+========================+
| predictions_ANERtestC10_Camel-ca-ner  |         31.36149 |   0.00000 | Significant (p < 0.05) |
+---------------------------------------+------------------+-----------+------------------------+
| predictions_ANERtestC10_Camel-msa-ner |          8.31635 |   0.00393 | Significant (p < 0.05) |
+---------------------------------------+------------------+-----------+------------------------+
| predictions_ANERtestC10_Camel-mix-ner |          1.07535 |   0.29974 | Not significant        |
+---------------------------------------+------------------+-----------+------------------------+
| predictions_ANERtestC10_Multilingual2 |          1.42730 |

#McNemar Test Weighted Voting


In [ ]:
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar
from tabulate import tabulate

def load_predictions(file_path):
    """
    Reads a .txt file with fixed headers:
    Token, Gold Tag, Predicted Tag
    Handles malformed lines safely.
    """
    print(f"\n📂 Loading: {file_path}")
    try:
        df = pd.read_csv(
            file_path,
            sep="\t",
            comment="#",
            skip_blank_lines=True,
            engine="python",
            on_bad_lines="warn"
        )
    except Exception as e:
        raise RuntimeError(f"❌ Error reading {file_path}: {e}")

    expected_cols = ["Token", "Gold Tag", "Predicted Tag"]
    if len(df.columns) > 3:
        print(f"⚠️ {file_path}: found {len(df.columns)} columns, trimming to 3.")
        df = df.iloc[:, :3]
        df.columns = expected_cols
    elif len(df.columns) == 3:
        df.columns = expected_cols
    else:
        raise ValueError(f"❌ {file_path} has invalid column structure ({len(df.columns)} columns).")

    before = len(df)
    df = df.dropna()
    after = len(df)
    if before != after:
        print(f"⚠️ Dropped {before - after} rows with missing values in {file_path}.")
    print(f"✅ Loaded {len(df)} valid rows from {file_path}")
    return df


# ==== الملفات ====
baseline_files = [
    "/content/predictions_ANERtestC10_Camel-ca-ner.txt",
    "/content/predictions_ANERtestC10_Camel-msa-ner.txt",
    "/content/predictions_ANERtestC10_Camel-mix-ner.txt",
    "/content/predictions_ANERtestC10_Multilingual2.txt"
]
ensemble_file = "/content/Compined_Predictions_Weighted_ANER.txt"


# ==== تحميل البيانات ====
ensemble = load_predictions(ensemble_file)

results_summary = []
tables = {}

# ==== اختبار McNemar ====
for file in baseline_files:
    baseline = load_predictions(file)

    # توحيد البيانات حسب Token لضمان التطابق
    merged = pd.merge(
        baseline,
        ensemble,
        on="Token",
        suffixes=("_base", "_ens"),
        how="inner"
    )

    print(f"🔄 Merged {len(merged)} rows between {file.split('/')[-1]} and ensemble")

    baseline_correct = merged["Gold Tag_base"] == merged["Predicted Tag_base"]
    ensemble_correct = merged["Gold Tag_ens"] == merged["Predicted Tag_ens"]

    a = sum(baseline_correct & ensemble_correct)
    b = sum(baseline_correct & ~ensemble_correct)
    c = sum(~baseline_correct & ensemble_correct)
    d = sum(~baseline_correct & ~ensemble_correct)

    table = [[a, b], [c, d]]
    tables[file] = table

    result = mcnemar(table, exact=False, correction=True)
    significance = "Significant (p < 0.05)" if result.pvalue < 0.05 else "Not significant"

    results_summary.append([
        file.replace(".txt", "").split("/")[-1],
        result.statistic,
        result.pvalue,
        significance
    ])

# ==== طباعة النتائج ====
print("\n=== Statistical Significance Results (McNemar Test) ===\n")
headers = ["Baseline Model", "Chi² Statistic", "p-value", "Significance"]
print(tabulate(results_summary, headers=headers, tablefmt="grid", floatfmt=".5f"))

print("\n=== Detailed 2x2 Comparison Tables ===")
for file, table in tables.items():
    print(f"\n{file.replace('.txt', '').split('/')[-1]}:")
    print(tabulate(
        table,
        headers=["Ensemble Correct", "Ensemble Wrong"],
        showindex=["Baseline Correct", "Baseline Wrong"],
        tablefmt="fancy_grid"
    ))



📂 Loading: /content/Compined_Predictions_Weighted_ANER.txt
⚠️ Dropped 2 rows with missing values in /content/Compined_Predictions_Weighted_ANER.txt.
✅ Loaded 14235 valid rows from /content/Compined_Predictions_Weighted_ANER.txt

📂 Loading: /content/predictions_ANERtestC10_Camel-ca-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-ca-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-ca-ner.txt
🔄 Merged 685599 rows between predictions_ANERtestC10_Camel-ca-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-msa-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-msa-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-msa-ner.txt
🔄 Merged 685607 rows between predictions_ANERtestC10_Camel-msa-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-mix-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERte

/tmp/ipython-input-3403134850.py:13: ParserWarning: Skipping line 4746: Expected 3 fields in line 4746, saw 4

  df = pd.read_csv(
/tmp/ipython-input-3403134850.py:13: ParserWarning: Skipping line 13249: Expected 3 fields in line 13249, saw 5

  df = pd.read_csv(



=== Statistical Significance Results (McNemar Test) ===

+---------------------------------------+------------------+-----------+------------------------+
| Baseline Model                        |   Chi² Statistic |   p-value | Significance           |
+=======================================+==================+===========+========================+
| predictions_ANERtestC10_Camel-ca-ner  |         33.14188 |   0.00000 | Significant (p < 0.05) |
+---------------------------------------+------------------+-----------+------------------------+
| predictions_ANERtestC10_Camel-msa-ner |          9.38793 |   0.00218 | Significant (p < 0.05) |
+---------------------------------------+------------------+-----------+------------------------+
| predictions_ANERtestC10_Camel-mix-ner |          1.49393 |   0.22161 | Not significant        |
+---------------------------------------+------------------+-----------+------------------------+
| predictions_ANERtestC10_Multilingual2 |          1.89776 |

#Boot Strap for Majority voting

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

def load_predictions(file_path):
    """
    Reads a .txt file with fixed headers:
    Token, Gold Tag, Predicted Tag
    Handles malformed lines safely.
    """
    print(f"\n📂 Loading: {file_path}")
    try:
        df = pd.read_csv(
            file_path,
            sep="\t",
            comment="#",
            skip_blank_lines=True,
            engine="python",
            on_bad_lines="warn"
        )
    except Exception as e:
        raise RuntimeError(f"❌ Error reading {file_path}: {e}")

    expected_cols = ["Token", "Gold Tag", "Predicted Tag"]
    if len(df.columns) > 3:
        print(f"⚠️ {file_path}: found {len(df.columns)} columns, trimming to 3.")
        df = df.iloc[:, :3]
        df.columns = expected_cols
    elif len(df.columns) == 3:
        df.columns = expected_cols
    else:
        raise ValueError(f"❌ {file_path} has invalid column structure ({len(df.columns)} columns).")

    before = len(df)
    df = df.dropna()
    after = len(df)
    if before != after:
        print(f"⚠️ Dropped {before - after} rows with missing values in {file_path}.")
    print(f"✅ Loaded {len(df)} valid rows from {file_path}")
    return df


# ==== الملفات ====
baseline_files = [
    "/content/predictions_ANERtestC10_Camel-ca-ner.txt",
    "/content/predictions_ANERtestC10_Camel-msa-ner.txt",
    "/content/predictions_ANERtestC10_Camel-mix-ner.txt",
    "/content/predictions_ANERtestC10_Multilingual2.txt"
]
ensemble_file = "/content/Compined_Predictions_Majority_ANER.txt"


# ==== تحميل البيانات ====
ensemble = load_predictions(ensemble_file)


def bootstrap_test(baseline_correct, ensemble_correct, n_bootstrap=1000, seed=42):
    """Bootstrap significance test on accuracy difference."""
    rng = np.random.default_rng(seed)
    n = len(baseline_correct)
    diffs = []
    for _ in range(n_bootstrap):
        idx = rng.integers(0, n, n)
        acc_base = np.mean(baseline_correct[idx])
        acc_ens = np.mean(ensemble_correct[idx])
        diffs.append(acc_ens - acc_base)
    diffs = np.array(diffs)
    ci_low, ci_high = np.percentile(diffs, [2.5, 97.5])
    p_value = 2 * min(np.mean(diffs <= 0), np.mean(diffs >= 0))  # two-sided
    return np.mean(diffs), ci_low, ci_high, p_value


# ==== اختبار Bootstrap ====
results_summary = []

for file in baseline_files:
    baseline = load_predictions(file)

    # توحيد البيانات حسب Token لضمان التطابق
    merged = pd.merge(
        baseline,
        ensemble,
        on="Token",
        suffixes=("_base", "_ens"),
        how="inner"
    )

    print(f"🔄 Merged {len(merged)} rows between {file.split('/')[-1]} and ensemble")

    baseline_correct = (merged["Gold Tag_base"] == merged["Predicted Tag_base"]).to_numpy()
    ensemble_correct = (merged["Gold Tag_ens"] == merged["Predicted Tag_ens"]).to_numpy()

    mean_diff, ci_low, ci_high, p_value = bootstrap_test(baseline_correct, ensemble_correct)

    significance = "Significant (p < 0.05)" if p_value < 0.05 else "Not significant"

    results_summary.append([
        file.replace(".txt", "").split("/")[-1],
        f"{mean_diff:.5f}",
        f"[{ci_low:.5f}, {ci_high:.5f}]",
        f"{p_value:.5f}",
        significance
    ])


# ==== طباعة النتائج ====
print("\n=== Bootstrap Significance Test Results ===\n")
headers = ["Baseline Model", "Mean Δ Accuracy", "95% CI", "p-value", "Significance"]
print(tabulate(results_summary, headers=headers, tablefmt="grid"))



📂 Loading: /content/Compined_Predictions_Majority_ANER.txt
⚠️ Dropped 2 rows with missing values in /content/Compined_Predictions_Majority_ANER.txt.
✅ Loaded 14235 valid rows from /content/Compined_Predictions_Majority_ANER.txt

📂 Loading: /content/predictions_ANERtestC10_Camel-ca-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-ca-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-ca-ner.txt
🔄 Merged 685599 rows between predictions_ANERtestC10_Camel-ca-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-msa-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-msa-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-msa-ner.txt
🔄 Merged 685608 rows between predictions_ANERtestC10_Camel-msa-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-mix-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERte

/tmp/ipython-input-2554979189.py:13: ParserWarning: Skipping line 4746: Expected 3 fields in line 4746, saw 4

  df = pd.read_csv(
/tmp/ipython-input-2554979189.py:13: ParserWarning: Skipping line 13249: Expected 3 fields in line 13249, saw 5

  df = pd.read_csv(


🔄 Merged 684641 rows between predictions_ANERtestC10_Multilingual2.txt and ensemble

=== Bootstrap Significance Test Results ===

+---------------------------------------+-------------------+---------------------+-----------+------------------------+
| Baseline Model                        |   Mean Δ Accuracy | 95% CI              |   p-value | Significance           |
+=======================================+===================+=====================+===========+========================+
| predictions_ANERtestC10_Camel-ca-ner  |           0.0003  | [0.00019, 0.00040]  |     0     | Significant (p < 0.05) |
+---------------------------------------+-------------------+---------------------+-----------+------------------------+
| predictions_ANERtestC10_Camel-msa-ner |           0.00013 | [0.00004, 0.00022]  |     0.008 | Significant (p < 0.05) |
+---------------------------------------+-------------------+---------------------+-----------+------------------------+
| predictions_ANERtestC

#Boot Strap for weighted voting

In [ ]:
import pandas as pd
import numpy as np
from tabulate import tabulate

def load_predictions(file_path):
    """
    Reads a .txt file with fixed headers:
    Token, Gold Tag, Predicted Tag
    Handles malformed lines safely.
    """
    print(f"\n📂 Loading: {file_path}")
    try:
        df = pd.read_csv(
            file_path,
            sep="\t",
            comment="#",
            skip_blank_lines=True,
            engine="python",
            on_bad_lines="warn"
        )
    except Exception as e:
        raise RuntimeError(f"❌ Error reading {file_path}: {e}")

    expected_cols = ["Token", "Gold Tag", "Predicted Tag"]
    if len(df.columns) > 3:
        print(f"⚠️ {file_path}: found {len(df.columns)} columns, trimming to 3.")
        df = df.iloc[:, :3]
        df.columns = expected_cols
    elif len(df.columns) == 3:
        df.columns = expected_cols
    else:
        raise ValueError(f"❌ {file_path} has invalid column structure ({len(df.columns)} columns).")

    before = len(df)
    df = df.dropna()
    after = len(df)
    if before != after:
        print(f"⚠️ Dropped {before - after} rows with missing values in {file_path}.")
    print(f"✅ Loaded {len(df)} valid rows from {file_path}")
    return df


# ==== الملفات ====
baseline_files = [
    "/content/predictions_ANERtestC10_Camel-ca-ner.txt",
    "/content/predictions_ANERtestC10_Camel-msa-ner.txt",
    "/content/predictions_ANERtestC10_Camel-mix-ner.txt",
    "/content/predictions_ANERtestC10_Multilingual2.txt"
]
ensemble_file = "/content/Compined_Predictions_Weighted_ANER.txt"


# ==== تحميل البيانات ====
ensemble = load_predictions(ensemble_file)


def bootstrap_test(baseline_correct, ensemble_correct, n_bootstrap=1000, seed=42):
    """Bootstrap significance test on accuracy difference."""
    rng = np.random.default_rng(seed)
    n = len(baseline_correct)
    diffs = []
    for _ in range(n_bootstrap):
        idx = rng.integers(0, n, n)
        acc_base = np.mean(baseline_correct[idx])
        acc_ens = np.mean(ensemble_correct[idx])
        diffs.append(acc_ens - acc_base)
    diffs = np.array(diffs)
    ci_low, ci_high = np.percentile(diffs, [2.5, 97.5])
    p_value = 2 * min(np.mean(diffs <= 0), np.mean(diffs >= 0))  # two-sided
    return np.mean(diffs), ci_low, ci_high, p_value


# ==== اختبار Bootstrap ====
results_summary = []

for file in baseline_files:
    baseline = load_predictions(file)

    # توحيد البيانات حسب Token لضمان التطابق
    merged = pd.merge(
        baseline,
        ensemble,
        on="Token",
        suffixes=("_base", "_ens"),
        how="inner"
    )

    print(f"🔄 Merged {len(merged)} rows between {file.split('/')[-1]} and ensemble")

    baseline_correct = (merged["Gold Tag_base"] == merged["Predicted Tag_base"]).to_numpy()
    ensemble_correct = (merged["Gold Tag_ens"] == merged["Predicted Tag_ens"]).to_numpy()

    mean_diff, ci_low, ci_high, p_value = bootstrap_test(baseline_correct, ensemble_correct)

    significance = "Significant (p < 0.05)" if p_value < 0.05 else "Not significant"

    results_summary.append([
        file.replace(".txt", "").split("/")[-1],
        f"{mean_diff:.5f}",
        f"[{ci_low:.5f}, {ci_high:.5f}]",
        f"{p_value:.5f}",
        significance
    ])


# ==== طباعة النتائج ====
print("\n=== Bootstrap Significance Test Results ===\n")
headers = ["Baseline Model", "Mean Δ Accuracy", "95% CI", "p-value", "Significance"]
print(tabulate(results_summary, headers=headers, tablefmt="grid"))



📂 Loading: /content/Compined_Predictions_Weighted_ANER.txt
⚠️ Dropped 2 rows with missing values in /content/Compined_Predictions_Weighted_ANER.txt.
✅ Loaded 14235 valid rows from /content/Compined_Predictions_Weighted_ANER.txt

📂 Loading: /content/predictions_ANERtestC10_Camel-ca-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-ca-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-ca-ner.txt
🔄 Merged 685599 rows between predictions_ANERtestC10_Camel-ca-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-msa-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERtestC10_Camel-msa-ner.txt.
✅ Loaded 14235 valid rows from /content/predictions_ANERtestC10_Camel-msa-ner.txt
🔄 Merged 685607 rows between predictions_ANERtestC10_Camel-msa-ner.txt and ensemble

📂 Loading: /content/predictions_ANERtestC10_Camel-mix-ner.txt
⚠️ Dropped 2 rows with missing values in /content/predictions_ANERte

/tmp/ipython-input-1264013803.py:13: ParserWarning: Skipping line 4746: Expected 3 fields in line 4746, saw 4

  df = pd.read_csv(
/tmp/ipython-input-1264013803.py:13: ParserWarning: Skipping line 13249: Expected 3 fields in line 13249, saw 5

  df = pd.read_csv(



=== Bootstrap Significance Test Results ===

+---------------------------------------+-------------------+---------------------+-----------+------------------------+
| Baseline Model                        |   Mean Δ Accuracy | 95% CI              |   p-value | Significance           |
+=======================================+===================+=====================+===========+========================+
| predictions_ANERtestC10_Camel-ca-ner  |           0.00031 | [0.00020, 0.00041]  |     0     | Significant (p < 0.05) |
+---------------------------------------+-------------------+---------------------+-----------+------------------------+
| predictions_ANERtestC10_Camel-msa-ner |           0.00014 | [0.00006, 0.00024]  |     0.006 | Significant (p < 0.05) |
+---------------------------------------+-------------------+---------------------+-----------+------------------------+
| predictions_ANERtestC10_Camel-mix-ner |           6e-05   | [-0.00004, 0.00015] |     0.24  | Not signifi